In [60]:
import nltk
import string
import spacy
from sacremoses import MosesTokenizer, MosesDetokenizer
from spacy import displacy
from collections import Counter
import truecase
import en_core_web_sm
nlp = en_core_web_sm.load()
from pprint import pprint
from unidecode import unidecode
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
import warnings
from tqdm.notebook import tqdm_notebook
from scipy.spatial import distance
import numpy as np
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from sentence_transformers import SentenceTransformer
from nltk.corpus import stopwords
import tensorflow as tf
import transformers
from sent2vec.vectorizer import Vectorizer as S2vectorizer
tqdm_notebook.pandas()
warnings.filterwarnings("ignore")

pd.set_option('display.max_colwidth', 0)
pd.set_option('display.max_columns', 0)

## NER Generation

In [41]:
pprint([(X.text, X.label_) for X in nlp('The political system of the Islamic Republic is based on the 1979 Constitution, and comprises several intricately connected governing bodies. The Leader of the Revolution ("Supreme Leader") is responsible for delineation and supervision of the general policies of the Islamic Republic of Iran. The Supreme Leader is Commander-in-Chief of the armed forces, controls the military intelligence and security operations, and has sole power to declare war or peace. The heads of the judiciary, state radio and television networks, the commanders of the police and military forces and six of the twelve members of the Guardian Council are appointed by the Supreme Leader. The Assembly of Experts elects and dismisses the Supreme Leader on the basis of qualifications and popular esteem.').ents])

[('the Islamic Republic', 'GPE'),
 ('1979', 'DATE'),
 ('Constitution', 'LAW'),
 ('the Islamic Republic of Iran', 'GPE'),
 ('six', 'CARDINAL'),
 ('twelve', 'CARDINAL'),
 ('the Guardian Council', 'ORG'),
 ('The Assembly of Experts', 'ORG')]


In [55]:
pprint([(X.text, X.label_) for X in nlp('german').ents])

[]


In [56]:
pprint([(X.text, X.label_) for X in nlp('German').ents])

[('German', 'NORP')]


In [13]:
def generate_ners(text):
    result = [unidecode(X.text.lower()) for X in nlp(text).ents]
    return result

In [ ]:
df = pd.read_csv('../data/SQuAD_csv.csv', encoding='utf-8').loc[:, ['context']]
df = df.drop_duplicates(subset=['context'])
df['context_ner'] = df['context'].progress_apply(generate_ners)
# df['context'] = df['context'].apply(lambda x: str(x).lower())

In [ ]:
df.head()

## Text Preprocessing

In [26]:
stop_words = stopwords.words("english")

def clean_normalcase_stop(text):
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\s{2,}', " ", text)
    text = unidecode(text)
    return text

def clean_normalcase_nostop(text):
    text = ' '.join([word for word in text.split() if word not in stop_words])
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\s{2,}', " ", text)
    text = unidecode(text)
    return text

def clean_lowercase_stop(text):
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\s{2,}', " ", text)
    text = unidecode(text)
    return text.lower()

def clean_lowercase_nostop(text):
    text = ' '.join([word for word in text.split() if word not in stop_words])
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\s{2,}', " ", text)
    text = unidecode(text)
    return text.lower()

In [34]:
df['cleaned_normalcase_stop'] = df['context'].progress_apply(clean_normalcase_stop)

In [35]:
df['cleaned_normalcase_nostop'] = df['context'].progress_apply(clean_normalcase_nostop)

In [36]:
df['cleaned_lowercase_stop'] = df['context'].progress_apply(clean_lowercase_stop)

In [37]:
df['cleaned_lowercase_nostop'] = df['context'].progress_apply(clean_lowercase_nostop)

In [40]:
df['ner_combined'] = df['context_ner'].apply(lambda x: ' '.join(x).strip('[').strip(']').replace(',',''))

In [41]:
df.head(10)

,context,context_ner,cleaned_normalcase_stop,cleaned_normalcase_nostop,cleaned_lowercase_stop,cleaned_lowercase_nostop,ner_combined
0,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles ""Crazy in Love"" and ""Baby Boy"".","[beyonce giselle knowles-carter, september 4, 1981, american, houston, texas, the late 1990s, r&b, destiny, mathew knowles, beyonce, 2003, five, 100, one, crazy in love, baby boy]",Beyonce Giselle Knowles Carter bi:'janseI bee YON say born September 4 1981 is an American singer songwriter record producer and actress Born and raised in Houston Texas she performed in various singing and dancing competitions as a child and rose to fame in the late 1990s as lead singer of R B girl group Destiny s Child Managed by her father Mathew Knowles the group became one of the world s best selling girl groups of all time Their hiatus saw the release of Beyonce s debut album Dangerously in Love 2003 which established her as a solo artist worldwide earned five Grammy Awards and featured the Billboard Hot 100 number one singles Crazy in Love and Baby Boy,Beyonce Giselle Knowles Carter bi:'janseI bee YON say born September 4 1981 American singer songwriter record producer actress Born raised Houston Texas performed various singing dancing competitions child rose fame late 1990s lead singer R B girl group Destiny s Child Managed father Mathew Knowles group became one world s best selling girl groups time Their hiatus saw release Beyonce s debut album Dangerously Love 2003 established solo artist worldwide earned five Grammy Awards featured Billboard Hot 100 number one singles Crazy Love Baby Boy,beyonce giselle knowles carter bi:'jansei bee yon say born september 4 1981 is an american singer songwriter record producer and actress born and raised in houston texas she performed in various singing and dancing competitions as a child and rose to fame in the late 1990s as lead singer of r b girl group destiny s child managed by her father mathew knowles the group became one of the world s best selling girl groups of all time their hiatus saw the release of beyonce s debut album dangerously in love 2003 which established her as a solo artist worldwide earned five grammy awards and featured the billboard hot 100 number one singles crazy in love and baby boy,beyonce giselle knowles carter bi:'jansei bee yon say born september 4 1981 american singer songwriter record producer actress born raised houston texas performed various singing dancing competitions child rose fame late 1990s lead singer r b girl group destiny s child managed father mathew knowles group became one world s best selling girl groups time their hiatus saw release beyonce s debut album dangerously love 2003 established solo artist worldwide earned five grammy awards featured billboard hot 100 number one singles crazy love baby boy,beyonce giselle knowles-carter september 4 1981 american houston texas the late 1990s r&b destiny mathew knowles beyonce 2003 five 100 one crazy in love baby boy
15,"Following the disbandment of Destiny's Child in June 2005, she released her second solo album, B'Day (2006), which contained hits ""Déjà Vu"", ""Irreplaceable"", and ""Beautiful Liar"". Beyoncé also ventured into acting, with a Golden Globe-nominated performance in Dreamgirls (2006), and starring roles in The Pink Panther (2006) and Obsessed (2009). Her marriage to rapper Jay Z 

In [71]:
df.to_pickle('cleaned_ner_23_mar.pkl')

## Read .pkl File

In [10]:
#Read the pickle file
df2 = pd.read_pickle('cleaned_ner_23_mar.pkl')

In [12]:
ner_vectorizer = TfidfVectorizer()
context_vectorizer = TfidfVectorizer()
ner_tfidf = ner_vectorizer.fit_transform(df2['ner_combined'])
context_tfidf = context_vectorizer.fit_transform(df2['cleaned_lowercase_nostop'])

In [67]:
' '.join([])

''

In [110]:
def get_ner_matching_ids(query):
    for_ner_matching = re.sub('[%s]' % re.escape(string.punctuation), '', query)
    for_ner_matching = re.sub('\s{2,}', " ", for_ner_matching).lower()
    for_ner_matching = ' '.join([word for word in for_ner_matching.split() if word not in stop_words])
    for_ner_matching = truecase.get_true_case(for_ner_matching)
    
    query_ners = ' '.join(generate_ners(for_ner_matching))
    if query_ners == '':
        return None
    print(f"query_ners: {query_ners}")
    
    ner_matching_ids = get_similar_doc(ner_vectorizer, ner_tfidf, query_ners)
    
    # to add: context tfidf
    
    return ner_matching_ids

In [112]:
# print(get_ner_matching_docs('when was donald trump elected'))
print(get_ner_matching_ids('What leader tried to unite all people considered themselves "German"'))

query_ners: german
[3444, 3453, 3454, 3456, 3457, 8879, 8881, 17800]


In [ ]:
# def 
#     q1 = ner
#     q2 = tfidf context
    
#     if overlap:
#         return overlap
#     else:
        

## TF-IF Context Querying

In [ ]:
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(df['cleaned_context'])

In [ ]:
print(tfidf.shape)

In [119]:
def get_similar_docs_ner(vectorizer, docs_tfidf, query):
    # returns list of doc_ids using NER
    for_ner_matching = re.sub('[%s]' % re.escape(string.punctuation), '', query)
    for_ner_matching = re.sub('\s{2,}', " ", for_ner_matching).lower()
    for_ner_matching = ' '.join([word for word in for_ner_matching.split() if word not in stop_words])
    for_ner_matching = truecase.get_true_case(for_ner_matching)
    
    query_ners = ' '.join(generate_ners(for_ner_matching))
    if query_ners == '':
        return []
    
    ner_matching_ids = get_similar_doc(ner_vectorizer, ner_tfidf, query_ners)
    return ner_matching_ids
    
    return ner_matching_ids
def get_similar_docs_ngrams():
    # returns list of doc_ids using ngram-tfidf

def get_similar_docs(tfidfvectorizer, docs_tfidf, query):
    """
    vectorizer: TfIdfVectorizer model
    docs_tfidf: tfidf vectors for all docs
    query: query doc

    return: doc with highest tf-idf cosine similarity
    """
    query_tfidf = tfidfvectorizer.transform([query])
    cosineSimilarities = cosine_similarity(query_tfidf, docs_tfidf).flatten()
    
    doc_ids = []
    for idx, val in enumerate(cosineSimilarities):
        if val >= 0.7:
            doc_ids.append(idx)
            
    if len(doc_ids) == 0:
            
    return doc_ids

def get_relevant_sentence(doc_ids, query):
    print(doc_ids)
    docs = df2.iloc[doc_ids, df2.columns.get_loc('context')].apply(lambda x: [i.strip() for i in x.split('.') if len(i)>1])
    sentences = []
    for doc in docs:
        for sentence in doc:
            sentences.append(sentence)

    vectorizer = S2vectorizer()
    vectorizer.bert([query] + sentences)
    vectors = vectorizer.vectors
    query_vec, other_vec = vectors[0], vectors[1:]
    
    print("----- Sentences Retrieved: -----")
    for idx, sentence in enumerate(sentences):
        print(f"{idx}. {sentence}")
    
    distances = distance.cdist([query_vec], other_vec, "cosine")[0]
    min_index = np.argmin(distances)
#     min_distance = distances[min_index]
#     max_similarity = 1 - min_distance
    return sentences[min_index]
    
def get_answer(query):
    query = re.sub('[%s]' % re.escape(string.punctuation), '', query)
    query = re.sub('\s{2,}', " ", query).lower()
    most_similar_doc_ids = get_similar_doc(vectorizer, tfidf, query)
    return get_relevant_sentence(most_similar_doc_ids, query)

def get_answer_ner(query):
    ner_matching_ids = get_ner_matching_ids(query)
    docs = df2.iloc[ner_matching_ids, df2.columns.get_loc('context')].apply(lambda x: [i.strip() for i in x.split('.') if len(i)>1])
    return docs

In [129]:
get_answer_ner('when did beyonce release her multi-platinum second album')

query_ners: multiplatinum second


11919    [Water splitting, in which water is decomposed into its component protons, electrons, and oxygen, occurs in the light reactions in all photosynthetic organisms, Some such organisms, including the alga Chlamydomonas reinhardtii and cyanobacteria, have evolved a second step in the dark reactions in which protons and electrons are reduced to form H2 gas by specialized hydrogenases in the chloroplast, Efforts have been undertaken to genetically modify cyanobacterial hydrogenases to efficiently synthesize H2 gas even in the presence of oxygen, Efforts have also been undertaken with genetically modified alga in a bioreactor]                                                                                                                                                                                                                                                                                                                                                                      
15783 

## Gigi

### infersent

In [115]:
def clean_lowercase_punc_nostop(text):
    text = ' '.join([word for word in text.split() if word not in stop_words])
#     text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\s{2,}', " ", text)
    text = unidecode(text)
    return text.lower()

In [116]:
df_infersent = pd.read_csv('../data/SQuAD_csv.csv', encoding='utf-8').loc[:, ['context']]
df_infersent = df_infersent.drop_duplicates(subset=['context'])

df_infersent['context'] = df_infersent['context'].apply(lambda x: str(x).lower())
df_infersent['cleaned_lowercase_punc_nostop'] = df['context'].progress_apply(clean_lowercase_punc_nostop)

In [127]:
df_infersent.head()

,context,cleaned_lowercase_punc_nostop
0,"beyoncé giselle knowles-carter (/biːˈjɒnseɪ/ bee-yon-say) (born september 4, 1981) is an american singer, songwriter, record producer and actress. born and raised in houston, texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of r&b girl-group destiny's child. managed by her father, mathew knowles, the group became one of the world's best-selling girl groups of all time. their hiatus saw the release of beyoncé's debut album, dangerously in love (2003), which established her as a solo artist worldwide, earned five grammy awards and featured the billboard hot 100 number-one singles ""crazy in love"" and ""baby boy"".","beyonce giselle knowles-carter (/bi:'jansei/ bee-yon-say) (born september 4, 1981) american singer, songwriter, record producer actress. born raised houston, texas, performed various singing dancing competitions child, rose fame late 1990s lead singer r&b girl-group destiny's child. managed father, mathew knowles, group became one world's best-selling girl groups time. hiatus saw release beyonce's debut album, dangerously love (2003), established solo artist worldwide, earned five grammy awards featured billboard hot 100 number-one singles ""crazy love"" ""baby boy""."
15,"following the disbandment of destiny's child in june 2005, she released her second solo album, b'day (2006), which contained hits ""déjà vu"", ""irreplaceable"", and ""beautiful liar"". beyoncé also ventured into acting, with a golden globe-nominated performance in dreamgirls (2006), and starring roles in the pink panther (2006) and obsessed (2009). her marriage to rapper jay z and portrayal of etta james in cadillac records (2008) influenced her third album, i am... sasha fierce (2008), which saw the birth of her alter-ego sasha fierce and earned a record-setting six grammy awards in 2010, including song of the year for ""single ladies (put a ring on it)"". beyoncé took a hiatus from music in 2010 and took over management of her career; her fourth album 4 (2011) was subsequently mellower in tone, exploring 1970s funk, 1980s pop, and 1990s soul. her critically acclaimed fifth studio album, beyoncé (2013), was distinguished from previous releases by its experimental production and exploration of darker themes.","following disbandment destiny's child june 2005, released second solo album, b'day (2006), contained hits ""deja vu"", ""irreplaceable"", ""beautiful liar"". beyonce also ventured acting, golden globe-nominated performance dreamgirls (2006), starring roles pink panther (2006) obsessed (2009). marriage rapper jay z portrayal etta james cadillac records (2008) influenced third album, am... sasha fierce (2008), saw birth alter-ego sasha fierce earned record-setting six grammy awards 2010, including song year ""single ladies (put ring it)"". beyonce took hiatus music 2010 took management career; fourth album 4 (2011) subsequently mellower tone, exploring 1970s funk, 1980s pop, 1990s soul. critically acclaimed fifth studio album, beyonce (2013), distinguished previous releases experimental production exploration darker themes."
27,"a self-described ""modern-day feminist"", beyoncé creates songs that are often characterized by themes of love, relationships, and monogamy, as well as female sexuality and empowerment. on stage, her dynamic, highly choreographed performances have led to critics hailing her as one of the best entertainers in contemporary popular music. throughout a career spanning 19 years, she has sold over 118 million records as a solo artist, and a further 60 million with destiny's child, making her one of the best-selling music artists of all time. she has won 20 grammy awards and is the most nominated woman in the award's history. the recording industry association of america recognized her as the top certified artist in america during the 2000s decade. in 2009, billboard named her the top radio songs artist of 

In [130]:
infersent_vectorizer = TfidfVectorizer()
# infersent_tfidf = infersent_vectorizer.fit_transform(df['cleaned_lowercase_nostop'])
infersent_tfidf = infersent_vectorizer.fit_transform(df_infersent['cleaned_lowercase_punc_nostop'])

def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

def get_similar_doc_infersent(tfidfvectorizer, docs_tfidf, query):
    """
    vectorizer: TfIdfVectorizer model
    docs_tfidf: tfidf vectors for all docs
    query: query doc

    return: doc with highest tf-idf cosine similarity
    """
    
    query_tfidf = tfidfvectorizer.transform([query])
    cosineSimilarities = cosine_similarity(query_tfidf, docs_tfidf).flatten()
    cosineSimilarities = np.array(cosineSimilarities)
    return cosineSimilarities.argsort()[-3:][::-1]

def get_sentences(doc_ids):
#     docs = df.iloc[doc_ids, df.columns.get_loc('cleaned_lowercase_nostop')].apply(lambda x: [i.strip() for i in x.split('.') if len(i)>1])
    docs = df.iloc[doc_ids, df_infersent.columns.get_loc('cleaned_lowercase_punc_nostop')].apply(lambda x: [i.strip() for i in x.split('.') if len(i)>1])
    sentences = []
    sentences = []
    for doc in docs:
        for sentence in doc:
            sentences.append(sentence)
    return sentences

def infersent(sentences, query_vec):
    similarity = []
    for sent in sentences:
        sim = cosine(query_vec, model.encode([sent])[0])
        similarity.append((sent, sim))

    return similarity
# .sort(key=lambda x: x[1], reverse=True)

    
def get_answer_infersent(query):
    query = re.sub('[%s]' % re.escape(string.punctuation), '', query)
#     query = re.sub('\s{2,}', " ", query).lower()
    most_similar_doc_ids = get_similar_doc_infersent(infersent_vectorizer, infersent_tfidf, query)
    print(df_infersent.iloc[most_similar_doc_ids, df_infersent.columns.get_loc('context')])
    sentences = get_sentences(most_similar_doc_ids)
    model.build_vocab(sentences, tokenize=True)
    query_vec = model.encode(query)[0]
    
    answers = infersent(sentences, query_vec)
#     print(sorted(answers, key=lambda x: x[1]))
    return sorted(answers, key=lambda x: x[1], reverse=True)


In [ ]:
# ! mkdir encoder
# ! curl -Lo encoder/infersent2.pkl https://dl.fbaipublicfiles.com/infersent/infersent2.pkl
  
# ! mkdir GloVe
# ! curl -Lo GloVe/glove.840B.300d.zip http://nlp.stanford.edu/data/glove.840B.300d.zip
# ! unzip GloVe/glove.840B.300d.zip -d GloVe/

In [123]:
from models import InferSent
import torch

V = 2
MODEL_PATH = 'encoder/infersent%s.pkl' % V
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

W2V_PATH = 'GloVe/glove.840B.300d.txt'
model.set_w2v_path(W2V_PATH)

In [131]:
get_answer_infersent("What kind of monarchy was formed under Queen Victoria")

16750    at certain times of the year, the queen mary 2, queen elizabeth and queen victoria may all visit southampton at the same time, in an event commonly called 'arrival of the three queens'.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
35150    through victoria's reign, the gradual establishment of a modern constitutional monarchy in britain continued. reforms 

[('certain times year queen mary 2 queen elizabeth queen victoria may visit southampton time event commonly called arrival three queens',
  0.03225669),
 ('victoria s reign gradual establishment modern constitutional monarchy britain continued reforms voting system increased power house commons expense house lords monarch 1867 walter bagehot wrote monarch retained the right consulted right encourage right warn victoria s monarchy became symbolic political placed strong emphasis morality family values contrast sexual financial personal scandals associated previous members house hanover discredited monarchy concept family monarchy burgeoning middle classes could identify solidified',
  -0.093323655),
 ('eleven days orsini s assassination attempt france victoria s eldest daughter married prince frederick william prussia london betrothed since september 1855 princess victoria 14 years old marriage delayed queen prince albert bride 17 queen albert hoped daughter son in law would liberalisin

In [133]:
get_answer_infersent("Where is Freedom Monument")

85934    the city is home to many monuments and memorials, most notably those along monument avenue. other monuments include the a.p. hill monument, the bill "bojangles" robinson monument in jackson ward, the christopher columbus monument near byrd park, and the confederate soldiers and sailors monument on libby hill. located near byrd park is the famous world war i memorial carillon, a 56-bell carillon tower. dedicated in 1956, the virginia war memorial is located on belvedere overlooking the river, and is a monument to virginians who died in battle in world war ii, the korean war, the vietnam war, the gulf war, the war in afghanistan, and the iraq war.                                                                                                                                                         
3381     the statue of liberty national monument and ellis island immigration museum are managed by the national park service and are in both the states of new york and new jersey. the

[("june 14 1987 5 000 people gathered freedom monument riga laid flowers commemorate anniversary stalin s mass deportation latvians 1941 first large demonstration baltic republics commemorate anniversary event contrary official soviet history authorities crack demonstrators encouraged larger demonstrations throughout baltic states next major anniversary august 23 molotov pact demonstration november 18 date latvia's independence 1918 november 18 1987 hundreds police civilian militiamen cordoned central square prevent demonstration freedom monument thousands lined streets riga silent protest regardless",
  -0.12536578),
 ('statue liberty national monument ellis island immigration museum managed national park service states new york new jersey joined harbor governors island national monument new york historic sites federal management manhattan island include castle clinton national monument federal hall national memorial theodore roosevelt birthplace national historic site general grant n

### trigrams tfidf

In [54]:
df = pd.read_csv('../data/SQuAD_csv.csv', encoding='utf-8').loc[:, ['context']]
df = df.drop_duplicates(subset=['context'])

df['context'] = df['context'].apply(lambda x: str(x).lower())
df['cleaned_lowercase_nostop'] = df['context'].progress_apply(clean_lowercase_nostop)

In [63]:
# Getting trigrams  
trigram_vectorizer = CountVectorizer(ngram_range = (3,3)) 
X1 = trigram_vectorizer.fit_transform(df['cleaned_lowercase_nostop'])  
features = (trigram_vectorizer.get_feature_names()) 
print("\n\nFeatures : \n", features) 
print("\n\nX1 : \n", X1.toarray()) 
  
# Applying TFIDF 
trigram_vectorizer = TfidfVectorizer(ngram_range = (3,3)) 
trigram_tfidf = trigram_vectorizer.fit_transform(df['cleaned_lowercase_nostop']) 
scores = (trigram_tfidf.toarray()) 
print("\n\nScores : \n", scores) 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)





X1 : 
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


Scores : 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [64]:
def get_similar_doc_trigram(tfidfvectorizer, docs_tfidf, query):
    query_tfidf = tfidfvectorizer.transform([query])
    cosineSimilarities = cosine_similarity(query_tfidf, docs_tfidf).flatten()
    cosineSimilarities = np.array(cosineSimilarities)
    return cosineSimilarities.argsort()[-3:][::-1]

def get_trigram_matching_docs(query):
    for_trigram_matching = re.sub('[%s]' % re.escape(string.punctuation), '', query)
    for_trigram_matching = re.sub('\s{2,}', " ", for_trigram_matching).lower()
    for_trigram_matching = ' '.join([word for word in for_trigram_matching.split() if word not in stop_words])
    
    trigram_matching_ids = get_similar_doc_trigram(trigram_vectorizer, trigram_tfidf, for_trigram_matching)
    
    return df.iloc[trigram_matching_ids, df.columns.get_loc('context')]

In [78]:
print(get_trigram_matching_docs("When did Beyonce leave Destiny's Child and become a solo singer"))

86816    kathmandu metropolitan city (kmc), in order to promote international relations has established an international relations secretariat (irc). kmc's first international relationship was established in 1975 with the city of eugene, oregon, united states. this activity has been further enhanced by establishing formal relationships with 8 other cities: motsumoto city of japan, rochester of the usa, yangon (formerly rangoon) of myanmar, xi'an of the people's republic of china, minsk of belarus, and pyongyang of the democratic republic of korea. kmc's constant endeavor is to enhance its interaction with saarc countries, other international agencies and many other major cities of the world to achieve better urban management and developmental programs for kathmandu.              
28025    according to the writer of luke, mary was a relative of elizabeth, wife of the priest zechariah of the priestly division of abijah, who was herself part of the lineage of aaron and so of the tribe of 

### bigram

In [86]:
# Getting trigrams  
bigram_vectorizer = CountVectorizer(ngram_range = (1,2)) 
bigram_X1 = bigram_vectorizer.fit_transform(df['cleaned_lowercase_nostop'])  
bigram_features = (bigram_vectorizer.get_feature_names()) 
print("\n\nFeatures : \n", bigram_features) 
print("\n\nX1 : \n", bigram_X1.toarray()) 
  
# Applying TFIDF 
bigram_vectorizer = TfidfVectorizer(ngram_range = (1,2)) 
bigram_tfidf = bigram_vectorizer.fit_transform(df['cleaned_lowercase_nostop']) 
bigram_scores = (bigram_tfidf.toarray()) 
print("\n\nScores : \n", bigram_scores) 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)





X1 : 
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


Scores : 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [87]:
def get_similar_doc_bigram(tfidfvectorizer, docs_tfidf, query):
    query_tfidf = tfidfvectorizer.transform([query])
    cosineSimilarities = cosine_similarity(query_tfidf, docs_tfidf).flatten()
    cosineSimilarities = np.array(cosineSimilarities)
    return cosineSimilarities.argsort()[-3:][::-1]

def get_bigram_matching_docs(query):
    for_bigram_matching = re.sub('[%s]' % re.escape(string.punctuation), '', query)
    for_bigram_matching = re.sub('\s{2,}', " ", for_bigram_matching).lower()
    for_bigram_matching = ' '.join([word for word in for_bigram_matching.split() if word not in stop_words])
    
    bigram_matching_ids = get_similar_doc_bigram(bigram_vectorizer, bigram_tfidf, for_bigram_matching)
    
    return df.iloc[bigram_matching_ids, df.columns.get_loc('context')]

In [92]:
print(get_bigram_matching_docs("Where did a protector put a notice because of Queen Victoria's lack of public appearances in March of 1864"))

70019    public notice is given through legal notices in newspapers, and communicated to state and county agencies within the species' area. foreign nations may also receive notice of a listing. a public hearing is mandatory if any person has requested one within 45 days of the published notice. "the purpose of the notice and comment requirement is to provide for meaningful public participation in the rulemaking process." summarized the ninth circuit court in the case of idaho farm bureau federation v. babbitt.                                                                                                                                                                                                                                                                        
34851    victoria's self-imposed isolation from the public diminished the popularity of the monarchy, and encouraged the growth of the republican movement. she did undertake her official government duties, yet chose to 

In [95]:
print(get_bigram_matching_docs("Who did Victoria try to convince Disraeli  to act against during the Russo-Turkish war"))

35079    between april 1877 and february 1878, she threatened five times to abdicate while pressuring disraeli to act against russia during the russo-turkish war, but her threats had no impact on the events or their conclusion with the congress of berlin. disraeli's expansionist foreign policy, which victoria endorsed, led to conflicts such as the anglo-zulu war and the second anglo-afghan war. "if we are to maintain our position as a first-rate power", she wrote, "we must ... be prepared for attacks and wars, somewhere or other, continually." victoria saw the expansion of the british empire as civilising and benign, protecting native peoples from more aggressive powers or cruel rulers: "it is not in our custom to annexe countries", she said, "unless we are obliged & forced to do so." to victoria's dismay, disraeli lost the 1880 general election, and gladstone returned as prime minister. when disraeli died the following year, she was blinded by "fast falling tears", and erected a memor